## PySpark 설치

In [ ]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 처리할 데이터 파일을 먼저 다운로드 받아온다

In [1]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/1800.csv

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [2]:
!ls -tl

'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
!head -5 1800.csv

'head'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


## 판다스 데이터프레임으로 처리해본다

In [5]:
import pandas as pd 

pd_df = pd.read_csv(
    "./data/1800.csv",
    names=["stationID", "date", "measure_type", "temperature"],
    usecols=[0, 1, 2, 3]
)

In [6]:
pd_df.head()

,stationID,date,measure_type,temperature
0,ITE00100554,18000101,TMAX,-75
1,ITE00100554,18000101,TMIN,-148
2,GM000010962,18000101,PRCP,0
3,EZE00100082,18000101,TMAX,-86
4,EZE00100082,18000101,TMIN,-135


In [7]:
# Filter out all but TMIN entries
pd_minTemps = pd_df[pd_df['measure_type'] == "TMIN"]

In [8]:
pd_minTemps.head()

,stationID,date,measure_type,temperature
1,ITE00100554,18000101,TMIN,-148
4,EZE00100082,18000101,TMIN,-135
6,ITE00100554,18000102,TMIN,-125
9,EZE00100082,18000102,TMIN,-130
11,ITE00100554,18000103,TMIN,-46


In [29]:
# Select only stationID and temperature
pd_stationTemps = pd_minTemps[["stationID", "temperature"]]

In [30]:
# Aggregate to find minimum temperature for every station
pd_minTempsByStation = pd_stationTemps.groupby(["stationID"]).min("temperature")
pd_minTempsByStation.head()

,temperature
stationID,
EZE00100082,-135
ITE00100554,-148


## Spark으로 처리해본다

In [11]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

# SparkConf로 SparkSession의 환경 설정
conf = SparkConf()
# application의 이름
conf.set("spark.app.name", "PySpark DataFrame #1")
# master 설정, local 모든 스레드를 가져오겠다
conf.set("spark.master", "local[*]")

# spark이란 이름으로 spark session 오브젝트 설정
spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

In [31]:
# 데이터프레임으로 로드
# long form : spark.read.format("csv").load("./data/1800.csv")
# short form : spark.read.csv("1800.csv")
# s3 path여도 가능하다.
df = spark.read.format("csv").load("./data/1800.csv") # spark.read.csv("1800.csv")

In [32]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [33]:
# spark 데이터프레임 로드하지만, toDF로 로드 해서, 컬럼 이름 지정
# 간단하게 컬럼의 이름을 지정해주는 것
df = spark.read.format("csv")\
    .load("./data/1800.csv")\
    .toDF("stationID", "date", "measure_type", "temperature", "_c4", "_c5", "_c6", "_c7")

In [35]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: string (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [38]:
# inferSchema를 true로 지정하면, spark에서 앞의 샘플을 추출해서 schema를 추측해서 지정
df = spark.read.format("csv")\
    .option("inferSchema", "true")\
    .load("./data/1800.csv")\
    .toDF("stationID", "date", "measure_type", "temperature", "_c4", "_c5", "_c6", "_c7")

In [40]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)



In [41]:
from pyspark.sql.types import StringType, IntegerType, FloatType
from pyspark.sql.types import StructType, StructField

# 제대로 schema를 정의하고 로드하기
# StructFiled , 컬럼명, 타입, True면 null 값이 가능하다
schema = StructType([ \
                     StructField("stationID", StringType(), True), \
                     StructField("date", IntegerType(), True), \
                     StructField("measure_type", StringType(), True), \
                     StructField("temperature", FloatType(), True)])

In [44]:
# df = spark.read.schema(schema).format("csv").load("1800.csv")
# 이렇게한 후에 로드하면, 앞에 4가지 컬럼에 대해서 지정한대로 들어간다.
df = spark.read.schema(schema).csv("./data/1800.csv")

In [45]:
df.printSchema()

root
 |-- stationID: string (nullable = true)
 |-- date: integer (nullable = true)
 |-- measure_type: string (nullable = true)
 |-- temperature: float (nullable = true)



In [46]:
# Filter out all but TMIN entries
minTemps = df.filter(df.measure_type == "TMIN")

In [47]:
minTemps.count()

730

In [48]:
# Column expression으로 필터링 적용
minTemps = df.where(df.measure_type == "TMIN")

In [49]:
minTemps.count()

730

In [52]:
# SQL expression으로 필터링 적용
minTemps = df.where("measure_type = 'TMIN'")

In [61]:
minTemps.count()

730

In [69]:
# Aggregate to find minimum temperature for every station
minTempsByStation = minTemps.groupBy("stationID").min("temperature")
minTempsByStation.show()

+-----------+----------------+
|  stationID|min(temperature)|
+-----------+----------------+
|ITE00100554|          -148.0|
|EZE00100082|          -135.0|
+-----------+----------------+



In [72]:
# Select only stationID and temperature
stationTemps = minTemps[["stationID", "temperature"]]

In [73]:
stationTemps.show(5)

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
|ITE00100554|     -125.0|
|EZE00100082|     -130.0|
|ITE00100554|      -46.0|
+-----------+-----------+
only showing top 5 rows



In [74]:
# 원하는 컬럼 나열 가능
stationTemps = minTemps.select("stationID", "temperature")

In [75]:
stationTemps.show(5)

+-----------+-----------+
|  stationID|temperature|
+-----------+-----------+
|ITE00100554|     -148.0|
|EZE00100082|     -135.0|
|ITE00100554|     -125.0|
|EZE00100082|     -130.0|
|ITE00100554|      -46.0|
+-----------+-----------+
only showing top 5 rows



In [82]:
# Collect, format, and print the results
# collect 하면 파이썬 쪽으로 받아온다.(리스트 형태로 받아옴)
results = minTempsByStation.collect()

In [86]:
for result in results:
    print(result[0] + "\t{:.2f}F".format(result[1]))

ITE00100554
ITE00100554	-148.00F
EZE00100082
EZE00100082	-135.00F


In [85]:
results[0] + result

Row(stationID='ITE00100554', min(temperature)=-148.0)

## Spark SQL로 처리해보기

In [79]:
df.createOrReplaceTempView("station1800")

In [80]:
results = spark.sql("""SELECT stationID, MIN(temperature)
FROM station1800
WHERE measure_type = 'TMIN'
GROUP BY 1""").collect()

In [81]:
# pyspark.sql.Row는 DataFrame의 레코드에 해당하며 필드별로 이름이 존재
for r in results:
    print(r)

Row(stationID='ITE00100554', min(temperature)=-148.0)
Row(stationID='EZE00100082', min(temperature)=-135.0)
